# Exploitation des jeux de données du hackathon IA & Mobilités 2024

**<!> Ce guide est destiné aux participants du hackathon et est valable pour les services instanciés dans le groupe "dlb-hackathon" <!>**

**<!> Les paramètres de connexion au stockage sont éphémères, la logique est de supprimer les services en fin de journée, __après sauvegarde de vos fichiers de travail__ <!>**

## Paramétrage de duckdb depuis les variables d'environnement

In [ ]:
# Paramétrage pour un appel Duckdb
%pip install duckdb
import duckdb as ddb
ddb.execute("SET s3_region='fr-central';")
ddb.execute("SET s3_url_style='path';")
ddb.execute("SET s3_endpoint='minio.data-platform-self-service.net';")
ddb.execute(f"SET s3_access_key_id='{os.environ["AWS_ACCESS_KEY_ID"]}' ;") # Aussi récupérable dans les paramètres "Valeurs de Helm" du service
ddb.execute(f"SET s3_secret_access_key='{os.environ["AWS_SECRET_ACCESS_KEY"]}';") # Aussi récupérable dans les paramètres "Valeurs de Helm" du service

## Revue des données

### Historique des messages d'info trafic

# Historique des messages d'info trafic

## Description générale
Ce dataset correspond à la compilation par IDFM des messages d'info trafic diffusés en temps réel au travers du service "https://prim.iledefrance-mobilites.fr/fr/apis/idfm-disruptions_bulk". Voir la documentation complète ici : https://prim.iledefrance-mobilites.fr/content/files/2024/03/-FR--Documentation-fonctionnelle-PRIM---Prise-en-main-de-l-API-Messages-Info-Trafic-RG-1.pdf.

Cet historique a été reconstitué par exploitation des messages conservés par pas de 30 minutes.

La période couverte va du 01/08/2024 au 12/11/2024, les périodes antérieures sont en cours de compilation et nous les espérons prêtes pour jeudi prochain !

## Applications possibles au hackathon
Ces données peuvent être utiles :
- pour étudier comment l'IA peut transformer les communications des opérateurs de transport pour les rendre plus accessibles;
- pour étudier les possibilités de mieux personnaliser l'expérience utilisateur,
- pour proposer des solutions de classification frugales permettant de mieux structurer ces données (défi 2),
- pour identifier des événements à impact et contribuer à la constitution d'un référentiels d'événements (défi 3)

## Infos techniques :
Données extraites depuis le datalake IDFM, couche gold :
```sql
WITH exploded_structs AS (
    SELECT
        *,
        UNNEST("application_periods") AS unnested_periods,
        UNNEST("tags") AS unnested_tags,
        unnest("application_patterns") AS unnested_patterns,
        unnest("messages_channel_types") AS unnested_channel_types,
        UNNEST("impacted_objects_impacted_section_routes") AS unnested_impacted_routes,
        UNNEST("impacted_objects_impacted_rail_section_routes") AS unnested_impacted_rail_routes
    FROM
        "{current_url}"
), date_fichier AS (
SELECT 
	cast(
    regexp_extract(url, '/(202[0-9])_', 1) ||      -- Année
    '-' || 
    regexp_extract(url, '/202[0-9]_([0-9]{2})_', 1) ||  -- Mois
    '-' || 
    regexp_extract(url, '/202[0-9]_[0-9]{2}_([0-9]{2})/', 1) ||  -- Jour
    ' ' || 
    regexp_extract(url, '/[0-9]{4}_[0-9]{2}_[0-9]{2}/([0-9]{2})/', 1) || -- Heure
    ':' || 
    regexp_extract(url, '/[0-9]{4}_[0-9]{2}_[0-9]{2}/[0-9]{2}/([0-9]{2})', 1)-- MINUTE
    AS timestamp
    ) 
    AS date_fichier
FROM (SELECT '{current_url}' url)
) 
FROM exploded_structs, date_fichier
SELECT 
	date_fichier,
    "id",
    "disruption_id",
    "impact_id",
	CAST(unnested_periods."begin" AS VARCHAR) AS period_begin,
    CAST(unnested_periods."end" AS VARCHAR) AS period_end,
    "status",
    "updated_at",
    string_agg(unnested_tags, ',') tags,
    "cause",
    "category",
    --"uri",
    --"disruption_uri",
    --"contributor",
    "severity.name",
    "severity.effect",
    "severity.color",
    "severity.priority",
    --string_agg(unnested_patterns, ',') patterns,
    "messages_text",
    --"messages_channel_id",
    --"messages_channel_content_type",
    --"messages_channel_name",
    --string_agg(unnested_channel_types, ',') channel_types,
    "impacted_objects_pt_object_id",
    "impacted_objects_pt_object_name",
    --"impacted_objects_impacted_section",
    --"impacted_objects_impacted_rail_section",
    --"impacted_objects_impacted_section_from_id",
    --"impacted_objects_impacted_section_from_name",
    --"impacted_objects_impacted_section_to_id",
    --"impacted_objects_impacted_section_to_name",
    --string_agg(unnested_impacted_routes, ',') impacted_routes,
    ---"impacted_objects_impacted_rail_section_from_id",
    --"impacted_objects_impacted_rail_section_from_name",
    --"impacted_objects_impacted_rail_section_to_id",
    --"impacted_objects_impacted_rail_section_to_name",
    --string_agg(unnested_impacted_rail_routes, ',') impacted_rail_routes
    date_fichier::time , CAST(period_begin AS timestamp)::time, date_diff('minute',date_fichier::time , CAST(period_begin AS timestamp)::time)
WHERE 
	(SUBSTRING(period_begin::varchar,0,11) = SUBSTRING(updated_at::varchar,0,11) )
	AND abs(date_diff('minute',date_fichier::time , CAST(period_begin AS timestamp)::time) ) <30
GROUP BY all
ORDER BY messages_text
```

In [ ]:
## Historique des messages d'info trafic
dataset = fs.ls("dlb-hackathon/datasets-diffusion/historique_info_trafic")

['dlb-hackathon/datasets-diffusion/historique_info_trafic/historique_disruptions_2024.parquet']

In [99]:
ddb.sql("""
(select (*) from "s3://dlb-hackathon/datasets-diffusion/historique_info_trafic/historique_disruptions_2024.parquet" where messages_text like '%%fortes pluies%%' )
""").to_df().head(5)

,date_fichier,id,disruption_id,impact_id,period_begin,period_end,status,updated_at,tags,cause,category,severity.name,severity.effect,severity.color,severity.priority,messages_text,impacted_objects_pt_object_id,impacted_objects_pt_object_name
0,2024-08-01 23:30:00,5e2cba06-504b-11ef-a594-0a58a9feac02,5e133176-504b-11ef-a594-0a58a9feac02,5e2cba06-504b-11ef-a594-0a58a9feac02,2024-08-01 23:14:47,2024-08-02 01:30:00,active,2024-08-01 23:16:50,Actualité,perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,<p>Le trafic est ralenti sur l'ensemble de la ...,line:IDFM:C01737,Transilien Train Transilien H
1,2024-09-26 18:30:00,e93b4d6c-7c22-11ef-a50c-0a58a9feac02,e93820ba-7c22-11ef-a50c-0a58a9feac02,e93b4d6c-7c22-11ef-a50c-0a58a9feac02,2024-09-26 18:15:37,2024-09-26 20:00:00,active,2024-09-26 18:18:05,"Actualité,Actualité",perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,<p>La gare de Orly Ville n'est pas desservie j...,line:IDFM:C01727,RER C
2,2024-09-26 19:00:00,078563fe-7c26-11ef-a193-0a58a9feac02,e93820ba-7c22-11ef-a50c-0a58a9feac02,078563fe-7c26-11ef-a193-0a58a9feac02,2024-09-26 18:38:34,2024-09-26 20:00:00,active,2024-09-26 18:40:24,"Actualité,Actualité",perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,<p>La gare de Orly Ville n'est pas desservie j...,line:IDFM:C01727,RER C
3,2024-10-08 17:30:00,4d2bed62-8589-11ef-a193-0a58a9feac02,4d2847f2-8589-11ef-a193-0a58a9feac02,4d2bed62-8589-11ef-a193-0a58a9feac02,2024-10-08 17:20:00,2024-10-09 23:59:00,active,2024-10-08 17:23:42,Actualité,perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,<p>09 octobre - Vigilance orange M&#233;t&#233...,line:IDFM:C01731,Transilien Train Transilien R
4,2024-10-09 02:00:00,7b183988-85cf-11ef-a193-0a58a9feac02,d3ea9a48-85ce-11ef-a193-0a58a9feac02,7b183988-85cf-11ef-a193-0a58a9feac02,2024-10-09 01:37:00,2024-10-09 23:55:00,active,2024-10-09 01:46:04,Actualité,perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,<p>Les fortes pluies de la d&#233;pression KIR...,line:IDFM:C01729,RER E


In [58]:
ddb.sql("""
summarize(select * from "s3://dlb-hackathon/datasets-diffusion/historique_info_trafic/historique_disruptions_2024.parquet" )
""").to_df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,date_fichier,TIMESTAMP,2024-01-23 09:30:00,2024-11-13 19:00:00,8547,None,None,2024-06-17 15:51:07.139191,2024-08-01 06:38:11.051164,2024-09-20 03:16:05.456373,533637,0.00
1,id,VARCHAR,00006838-6c61-11ef-ab99-0a58a9feac02,ffff344e-6621-11ef-9ca2-0a58a9feac02,116032,None,None,None,None,None,533637,0.00
2,disruption_id,VARCHAR,00008c30-ba89-11ee-9d9d-0a58a9feac02,fffe2ed4-2411-11ef-b7d8-0a58a9feac02,94687,None,None,None,None,None,533637,0.00
3,impact_id,VARCHAR,00006838-6c61-11ef-ab99-0a58a9feac02,ffff344e-6621-11ef-9ca2-0a58a9feac02,116032,None,None,None,None,None,533637,0.00
4,period_begin,VARCHAR,2023-01-10 10:50:00,2024-11-13 18:51:35,55754,None,None,None,None,None,533637,0.00
5,period_end,VARCHAR,2023-03-31 17:00:00,2031-12-31 16:00:00,48947,None,None,None,None,None,533637,0.00
6,status,VARCHAR,active,past,2,None,None,None,None,None,533637,0.00
7,updated_at,TIMESTAMP_NS,2023-01-10 11:07:17,2024-11-13 18:56:44,111863,None,None,2024-05-16 13:00:07.215257,2024-07-09 16:28:23.38179,2024-09-03 23:32:04.993981,533637,0.00
8,tags,VARCHAR,Actualité,"Masquée,Masquée,Masquée",48,None,None,None,None,None,533637,55.64
9,cause,VARCHAR,information,travaux,3,None,None,None,None,None,533637,0.00


## Recherches d'itinéraires agrégées

# Recherches d'itinéraires agrégées

## Description générale
Ce dataset correspond à la compilation par IDFM des informations essentielles des recherches d'itinéraires faites via le calculateur d'ininéraire ouvert au public et utilisé au sein de l'application IDFM.
Lien vers l'API d'origine : https://prim.iledefrance-mobilites.fr/fr/apis/idfm-navitia-general-v2
Documentation associée : https://prim.iledefrance-mobilites.fr/content/files/2024/04/-FR--Documentation-fonctionnelle-PRIM---Prise-en-main-des-API-Navitia-1.pdf

Agrégation réalisée : 
- date_requete
- date_recherchee_aaaa_mm_jj
- heure_recherchee_hh
- requete_deplacement_type_de_service_recommande
- requete_point_arrivee_iris
- nb_requetes
- requete_point_depart_iris **Cette donnée est masquée dès lors qu'elle concerne moins de 5 requêtes.**

La période couverte va du 01/08/2024 au 31/10/2024, les périodes antérieures sont en cours de compilation et nous les espérons prêtes pour jeudi prochain !

## Applications possibles au hackathon
Ces données peuvent être utiles :
- pour identifier des événements à impact et contribuer à la constitution d'un référentiels d'événements,
- pour établir un état des lieux indicatif de la demande et identifier des tendances,
- pour établir des corrélations avec des événements externes et améliorer les prévisions.

## Infos techniques :
Données extraites depuis le datalake IDFM, couche gold :
```sql
WITH agregat_complet AS 
(
	SELECT 
		SUBSTRING(requete_date::varchar, 0, 11) date_requete, -- jour de la requête
		count(DISTINCT id_requete) nb_requetes, -- nombre de requêtes
		SUBSTRING(requete_date_recherchee::varchar, 0, 11) date_recherchee_aaaa_mm_jj, -- jour du trajet recherché
		SUBSTRING(requete_date_recherchee::varchar, 11, 3) heure_recherchee_hh, -- heure du trajet recherché
		requete_deplacement_type_de_service_recommande, -- type de service recommandé, filtré sur 'Transport public'
		CASE WHEN requete_date_recherchee_est_au_depart = '-1' THEN requete_point_arrivee_iris ELSE requete_point_depart_iris END requete_point_depart_iris, -- zone de départ à la maille de l'iris
		CASE WHEN requete_date_recherchee_est_au_depart = '-1' THEN requete_point_depart_iris ELSE requete_point_arrivee_iris END requete_point_arrivee_iris, -- zone d'arrivée à la maille de l'iris
	FROM [fichiers RITI]
	WHERE requete_deplacement_type_de_service_recommande = 'publicTransport' -- seules les recommandations de 'Transport public' sont retenues
	GROUP BY ALL
) 
SELECT * EXCLUDE(nb_requetes,requete_point_depart_iris), 
	sum(nb_requetes) nb_requetes,
	CASE WHEN nb_requetes >4 THEN requete_point_depart_iris ELSE '99999999' END requete_point_depart_iris -- Lorsqu'il y a moins de 5 requetes, l'IRIS de départ n'est pas donné
FROM agregat_complet
GROUP BY ALL
ORDER BY 1,2
```

In [62]:
## Recherches d'itinéraires agrégées
fs.ls("dlb-hackathon/datasets-diffusion/recherches_itineraires_agregees")

['dlb-hackathon/datasets-diffusion/recherches_itineraires_agregees/.keep',
 'dlb-hackathon/datasets-diffusion/recherches_itineraires_agregees/HACKATHON_DATASET_RITI.ipynb',
 'dlb-hackathon/datasets-diffusion/recherches_itineraires_agregees/recherches_itineraires_agregees_8-9-10_2024.parquet',
 'dlb-hackathon/datasets-diffusion/recherches_itineraires_agregees/requete_source.sql']

In [63]:
ddb.sql("""
select * from "s3://dlb-hackathon/datasets-diffusion/recherches_itineraires_agregees/recherches_itineraires_agregees_8-9-10_2024.parquet"
""").to_df()

,date_requete,date_recherchee_aaaa_mm_jj,heure_recherchee_hh,requete_deplacement_type_de_service_recommande,requete_point_arrivee_iris,nb_requetes,requete_point_depart_iris
0,2024-08-01,2024-07-30,22,publicTransport,774640101,2.0,99999999
1,2024-08-01,2024-07-30,22,publicTransport,751166177,1.0,99999999
2,2024-08-01,2024-07-30,22,publicTransport,940520102,1.0,99999999
3,2024-08-01,2024-07-30,22,publicTransport,930480301,2.0,99999999
4,2024-08-01,2024-07-31,06,publicTransport,953060107,1.0,99999999
...,...,...,...,...,...,...,...
19144295,2024-10-31,2024-11-28,14,publicTransport,751155807,1.0,99999999
19144296,2024-10-31,2024-11-28,07,publicTransport,774490000,8.0,930080111
19144297,2024-10-31,2024-11-28,11,publicTransport,930730101,1.0,99999999
19144298,2024-10-31,2024-11-28,07,publicTransport,751176505,1.0,99999999


## Réclamations anonymisées

# Réclamations anonymisées

## Description générale
Ce dataset correspond à la compilation par IDFM des réclamations adressées par les usagers via ce formulaire en ligne : https://www.iledefrance-mobilites.fr/aide-et-contacts/nous-ecrire

Le dataset comprend notamment :
- la date de la demande, 
- le texte de la demande anonymisé, **n'hésitez pas à nous signaler tout éventuel défaut d'anonymisation**
- et sa classification par les services d'IDFM.

La période couverte va du 01/08/2024 au 31/10/2024, les périodes antérieures sont en cours de compilation et nous les espérons prêtes pour jeudi prochain !

## Applications possibles au hackathon
Ces données peuvent être utiles :
- pour identifier des événements à impact et contribuer à la constitution d'un référentiels d'événements,
- pour établir un état des lieux indicatif de la demande et identifier des tendances,
- pour établir des corrélations avec des événements externes et améliorer les prévisions.

## Infos techniques :
Données extraites depuis le datalake IDFM, couche gold :
```sql
SELECT id,"Date de la demande" date_demande,"Canal de la demande" canal_demande,
regexp_replace(regexp_replace(regexp_replace(regexp_replace(Texte_anonyme, 'emerciements(.|\n)*$', 'emerciements'), 'istinguée(.|\n)*$', 'istinguée'), 'ordialement(.|\n)*$', 'ordialement'), 'alutations(.|\n)*$', 'alutations') texte_anonymise, 
"Motif 1" classification_services_idfm
FROM [source]
WHERE "Canal de la demande" ='Formulaire Web'
AND NOT (
lower(Texte_anonyme) LIKE '%rue%' -- 316 occurences
OR 
lower(Texte_anonyme) LIKE '%arnaque%' -- 62 occurences
OR 
lower(Texte_anonyme) LIKE '%secours%' -- 6 occurences
OR
lower(Texte_anonyme) LIKE '%rgpd%' -- 3 occurences
OR
lower(Texte_anonyme) LIKE '%cnil%' -- 1 occurences
OR
lower(Texte_anonyme) LIKE '%dpo%' -- 1 occurences
OR
lower(Texte_anonyme) LIKE '%détresse%' -- 1 occurences
)
GROUP BY all;
```

In [64]:
## Recherches d'itinéraires agrégées
fs.ls("dlb-hackathon/datasets-diffusion/reclamations_formulaire_web")

['dlb-hackathon/datasets-diffusion/reclamations_formulaire_web/.keep',
 'dlb-hackathon/datasets-diffusion/reclamations_formulaire_web/idfm_reclamations_formulaire_web.parquet']

In [66]:
ddb.sql("""
select * from 's3://dlb-hackathon/datasets-diffusion/reclamations_formulaire_web/idfm_reclamations_formulaire_web.parquet'
""").to_df().sample(5)

,id,date_demande,canal_demande,texte_anonymise,classification_services_idfm
11532,43055,2024-03-29 08:38:00,Formulaire Web,Bonjour votre application ne me propose pas d’...,Tarification Abonnement > Support Canal Mobile...
10767,32868,2023-06-14 19:37:00,Formulaire Web,Bonjour \n Mes parents ont reçu ce document p...,Autres > Compétence IDFM
16883,50495,2024-09-16 20:46:00,Formulaire Web,"Bonjour, jeudi matin 12 septembre j’ai rejoins...",Tarification Abonnement > Tickets et billets >...
5151,22981,2022-12-28 11:30:00,Formulaire Web,Bonjour \n j ai besoin d'un document PDF atte...,SAV > Attestation de paiement
13836,42824,2024-03-23 13:38:00,Formulaire Web,Bonjour j'ai payée mon navigo mais la carte n'...,Tarification Abonnement > Support Canal Mobile...


# Historique des courses Transilien lignes H, J, L 2022 - 2023

# Historique transilien

## Description générale

Ce dataset correspond à la compilation par la SNCF des données relatives aux passages des trains des lignes transilien J, L et H.

Le dataset comprend notamment :
- Parcours théoriques et réalisés
- Ventilation des données de comptages annuels de référence sur 3 types de jour
- Capacité des trains

Voir la documentation des champs : https://minio.data-platform-self-service.net/dlb-hackathon/datasets-diffusion/courses_transilien_H_J_L_2022_2023/Documentation_vf.xlsx

La période couverte est 2022 - 2023.

## Applications possibles au hackathon
Ces données peuvent être utiles :
- pour établir des diagnostics et modèles de prévisions territorialisés notamment sur la communauté GPSeO;
- pour établir des modèles de prévision d'affluence, 
- pour identifier des incidents et les corréler à des événements externes,
- pour identifier des mécanismes de report de trafic etc.

In [67]:
ddb.sql("""
select * from 's3://dlb-hackathon/datasets-diffusion/courses_transilien_H_J_L_2022_2023/Course_et_comptage_Ligne_J_2023.csv'
""").to_df().sample(5)

,column00,Ligne,Année,Date,Jour,Numéro_marche_realisée,Gare,Rang_réalisé,Origine,Terminus,...,Capacité_assise,Capacité_totale,Taux_occupation_assis,Taux_occupation_debout,Composition_théorique,Nombre_voitures_théorique,Nombre_portes_théorique,Code_gare,Numero_marche_théorique,Rang_théorique
64754,5120481,J,2023,2023-01-27,JOB,131024.0,Conflans-Ste-Honorine Bâtiment Voyageurs,1.0,BOISSY-L'AILLERIE,PARIS-SAINT-LAZARE,...,792.0,1452.0,0.087466,0.160354,2 Z50000 - 7 caisses - 1452 places,14.0,14.0,CSH,131024,9.0
445094,5610508,J,2023,2023-05-03,JOB,137645.0,Paris-St-Lazare Bâtiment Voyageurs,1.0,PARIS-SAINT-LAZARE,ERMONT-EAUBONNE,...,792.0,1452.0,0.282369,0.517677,2 Z50000 - 7 caisses - 1452 places,14.0,14.0,PSL,137645,1.0
119363,5190186,J,2023,2023-02-21,JOB,136809.0,Chanteloup-les-Vignes *00*,24.0,PARIS-SAINT-LAZARE,MANTES-LA-JOLIE,...,792.0,1452.0,0.069559,0.127525,2 Z50000 - 7 caisses - 1452 places,14.0,14.0,CVF,136809,24.0
1084607,6538890,J,2023,2023-12-05,JOB,137032.0,Le Val-d'Argenteuil Bâtiment Voyageurs,24.0,GISORS,PARIS-SAINT-LAZARE,...,792.0,1452.0,0.357438,0.655303,2 Z50000 - 7 caisses - 1452 places,14.0,14.0,VDA,137032,24.0
260763,5369631,J,2023,2023-03-27,JOB,136558.0,Herblay Bâtiment Voyageurs,9.0,PONTOISE,PARIS-SAINT-LAZARE,...,792.0,1452.0,0.095730,0.175505,2 Z50000 - 7 caisses - 1452 places,14.0,14.0,HRY,136558,9.0


## Equipements en stations RATP

# Equipements des stations RATP

## Description générale

Ce dataset correspond à une extraction du projet en cours de recensement des équipements et espaces en gare mené par la RATP et Ile de France Mobilités. 
Les finalités sont multiples, de la meilleure gestion des matériels aux possibilités de de compléter l’information voyageur par du cheminement « indoor ».

Le dataset comprend :
-	Localisation des escaliers et escaliers mécaniques restreints aux espaces publics
-	Localisation des interphones d’appel voyageur, et localisation textuelle indicative
-	Localisation des sanitaires publics recensés avec étage associé

## Applications possibles au hackathon
Ces données peuvent être utiles pour les thématiques suivantes :
-	Améliorer recherche des itinéraires accessibles / Défi 1 - Accessibilité
-	Améliorer accès Open DATA IDFM / Défi 2 - Boîte à outils IA
-	Améliorer production Open DATA IDFM / Défi 2 - Boîte à outils IA
-	Enrichissement chatbotFAQ IDFM avec les données de mobilités d'IDFM / Défi 2 - Boîte à outils IA
-	Suggestion d'itinéraire automatisée / Défi 4 - Personnaliser l'expérience utilisateur
-	Enrichissement chatbotFAQ IDFM / Défi 4 - Personnaliser l'expérience utilisateur
-	Aider à pré-remplir le formulaire de réclamation / Défi 4 - Personnaliser l'expérience utilisateur

## Informations techniques

-	Escaliers :
```sql
-- Localisation des escaliers et escaliers mécaniques
select b.nom, id_escalier,type_esc,nb_marche,rampe,a.etage, c.etage, c.transport , c.compatibilite_pmr  , case when c.geom is not null then c.geom else st_centroid(a.geom) end localisation
from ari_escalier a
inner join ari_station b on (a.id_station=b.id_station)
inner join ari_espace d on (a.id_espace=d.id_espace)
left outer join ari_equipement c on (a.id_equipement = c.id_equipement)
where d.public_prive ='PUBLIC';
```
-	Interphones :
```sql
-- Localisation des interphones d'appel
select b.nom, a.type_equipement , a.localisant , a.geom localisation
from ari_equipement a
inner join ari_station b on (a.id_station=b.id_station)
inner join ari_espace d on (a.id_espace=d.id_espace)
where type_equipement = 'Interphone Appel Voyageur'
and d.public_prive ='PUBLIC';
```
-	Sanitaires :
```sql
-- Localisation des sanitaires publics
select b.nom nom_station, c.numero etage, st_centroid(a.geom) localisation
from ariane.ari_espace a
inner join ari_station b on (a.id_station=b.id_station)
inner join ari_etage c on (a.id_etage=c.id_etage)
where a.libel_dest_espace  = 'Sanitaires publiques'
and public_prive ='PUBLIC';
```

In [68]:
ddb.sql("""
select * from 's3://dlb-hackathon/datasets-diffusion/equipements_gares_ratp/ratp_localisation_sanitaires.csv'
""").to_df().sample(5)

,nom_station,etage,localisation
11,Vincennes,0,POINT (1658421.217620417 8183180.892164317)
33,Bibliothèque François Mitterrand,-3,POINT (1654302.9398312196 8181273.455817694)
68,Saint-Germain-en-Laye,-3,POINT (1633625.5860513267 8189095.349090577)
41,Grands Boulevards,-2,POINT (1651832.069683969 8185917.753758978)
40,Olympiades,-3,POINT (1653452.7765972856 8180939.269706332)


In [69]:
ddb.sql("""
select * from 's3://dlb-hackathon/datasets-diffusion/equipements_gares_ratp/ratp_localisation_escaliers.csv'
""").to_df().sample(5)

,nom,id_escalier,type_esc,nb_marche,rampe,etage,etage_1,transport,compatibilite_pmr,localisation
5182,Villejuif Léo Lagrange,1872054,EF,10.0,NON,-1,NaN,None,None,POINT (1653496.321556646 8176525.457920052)
532,Les Sablons,1588525,EF,10.0,OUI,-2,NaN,None,None,POINT (1646586.8594463414 8187050.15582005)
2526,Nation,1670836,EF,9.0,OUI,-2,NaN,None,None,POINT (1655625.790608251 8183359.198816701)
2584,Nation,1679077,EF,20.0,NON,-4,NaN,None,None,POINT (1655725.4485321096 8183271.944217526)
7318,Alma - Marceau,100068842,EF,20.0,OUI,0,NaN,None,None,POINT (1648707.6373959652 8185158.157877283)


In [70]:
ddb.sql("""
select * from 's3://dlb-hackathon/datasets-diffusion/equipements_gares_ratp/ratp_localisation_interphones.csv'
""").to_df().sample(5)

,nom,type_equipement,localisant,localisation
1026,Nation,Interphone Appel Voyageur,None,POINT (-7894427.117987629 2352880.4556147037)
585,Orsay Ville,Interphone Appel Voyageur,CR214,POINT (1639517.0868000123 8166658.344100981)
1005,Nation,Interphone Appel Voyageur,None,POINT (-7894422.848987628 2352852.4555147)
3442,Crimée,Interphone Appel Voyageur,ZC Milieu de quai 1,POINT (1654333.6177000105 8188050.418300919)
3580,Hoche,Interphone Appel Voyageur,None,POINT (1656168.8336000068 8188070.0984009225)


# Datasets liés aux mesures en faveur des nouvelles mobilités en communauté GPSEO

# Datasets liés aux mesures en faveur des nouvelles mobilités en communauté GPSEO
Grand Paris Seine et Oise

## Description générale

Un ensemble de 5 fichiers cartographiques (formats parquet et shapefile), correspondant :   | 
•	AMENAGEMENTS_CYCL_COMPL_CU :  sas vélos et cedez le passage, ce sont des aménagements de la voirie ayant pour objectifs d'améliorer la qualité des circulations douces que ce soir en terme de fluidité ou de sécurité.
•	IRVE_CU :  ce sont les bornes de recharges pour véhicules électriques sur l'espace public (donc disponibles a tous) installées sur fincancement de la communauté urbaine pour améliorer le maillage de ces infrastructures en complément des d'autres initiatives publiques ou privées.
•	MOBILITE_VELO_STATIONNEMENT_CU : ce sont des équipements de diverses natures permettant de stationner des vélos. Ces équipement sont mis en place par la collectivité afin d'améliorer ce maillage sur le territoire et ainsi facilité les déplacements en vélo et donc d'accompagner ce mode de déplacement doux.
•	SDC_PREFAISABILITE_CU : projet de schéma directeur cyclable CU et Département des Yvelines : ce sont les voies, itinéraires qui seront aménagées, ou réaménagés pour une utilisation cyclable par la CU et le département des yvelines. L'objectif est de développer ces itinéraires et ainsi favoriser les déplacements cyclables en améliorant l'expérience et la sécurité.
•	STATIONNEMENT_TROTTINETTE_CU : Les sationnement trotitnettes mises en place par la CU toujours pour faciliter les déplacements doux sur le territoire.

Des précisions complémentaires sont disponibles dans ce fichier : dlb-hackathon/datasets-diffusion/donnees_territoire_GPSEO/DOC HACKATHON IDFM.xlsx

In [71]:
ddb.sql("""
select * from 's3://dlb-hackathon/datasets-diffusion/donnees_territoire_GPSEO/amenagements_cyclables/AMENAGEMENTS_CYCL_COMPL_CU.parquet'
""").to_df().sample(5)

,objectid,typologie,gestionnai,moa,annee,commentair,nom_commun,code_insee,cout_trava,date_modif,longueur,geom
22,3203,Cédez-le-passage cycliste au feu rouge,CU GPSEO,cu gpseo,2024,None,VILLENNES-SUR-SEINE,None,None,1970-01-01,0,"[1, 1, 0, 0, 0, 7, 206, 115, 244, 130, 189, 25..."
27,3601,SAS vélo,CU GPSEO,cu gpseo,2016,None,CONFLANS-SAINTE-HONORINE,None,None,2024-10-31,0,"[1, 1, 0, 0, 0, 137, 242, 87, 229, 157, 249, 0..."
18,3201,SAS vélo,CU GPSEO,cu gpseo,2024,None,VILLENNES-SUR-SEINE,None,None,1970-01-01,0,"[1, 1, 0, 0, 0, 36, 175, 169, 232, 6, 189, 255..."
29,4001,SAS vélo,CU GPSEO,cu gpseo,None,None,CONFLANS-SAINTE-HONORINE,None,None,2024-10-31,0,"[1, 1, 0, 0, 0, 173, 116, 104, 21, 126, 173, 0..."
15,2003,None,None,None,None,None,None,None,None,1970-01-01,0,"[1, 1, 0, 0, 0, 16, 223, 89, 160, 35, 184, 252..."


In [72]:
ddb.sql("""
select * from 's3://dlb-hackathon/datasets-diffusion/donnees_territoire_GPSEO/bornes_recharges/IRVE_CU.parquet'
""").to_df().sample(5)

,objectid,adresse_ir,ville,nom_statio,geom
5,9,13 Rue Du Lot,BUCHELAY,BUCHELAY - Rue Du Lot,"[1, 1, 0, 0, 0, 30, 84, 174, 21, 80, 200, 250,..."
56,61,117 Place Du General De Gaulle,VERNEUIL-SUR-SEINE,VERNEUIL-SUR-SEINE - Place Du General De Gaulle,"[1, 1, 0, 0, 0, 116, 54, 11, 253, 47, 183, 255..."
97,491,Rue des Monts Chauvets,Poissy,POISSY - Coudraie,"[1, 1, 0, 0, 0, 182, 89, 237, 68, 88, 26, 0, 6..."
93,891,Avenue Lénine,Achères,ACHERES - Lénine,"[1, 1, 0, 0, 0, 60, 192, 235, 13, 91, 129, 0, ..."
15,23,Rue Pasteur,FONTENAY-SAINT-PERE,FONTENAY-SAINT-PERE - Rue Pasteur,"[1, 1, 0, 0, 0, 119, 181, 62, 21, 215, 2, 252,..."


In [73]:
ddb.sql("""
select * from 's3://dlb-hackathon/datasets-diffusion/donnees_territoire_GPSEO/projet_schema_directeur_cyclable/SDC_PREFAISABILITE_CU.parquet'
""").to_df().sample(5)

,code_l,code_s,code_it,code_comm,am_exist,am_ms12,am_artif,am_rd,am_long,am_temp,...,commenta_1,intégrati,linéaire_,f_element_,linéaire1,linéair_1,objectid,nature_voi,st_length_,geom
13,CL02,CL0203-04-05,CL,Hardricourt,None,None,0.0,0.0,816.67401,LT,...,None,None,0.0,0.0,816.7,0.0,94,None,816.606310,"[1, 2, 0, 0, 0, 22, 0, 0, 0, 13, 19, 16, 194, ..."
56,CL06,CL0605,CL,Bouaffle,Absence,Zone 20,0.0,0.0,955.10000,CT,...,None,None,955.1,1.0,0.0,303.0,221,None,273.161059,"[1, 2, 0, 0, 0, 3, 0, 0, 0, 31, 27, 245, 176, ..."
33,CL06,CL0603,CL,Bouaffle,Absence,Zone 20,0.0,0.0,116.10000,CT,...,None,None,116.1,0.0,0.0,116.1,189,None,116.052530,"[1, 2, 0, 0, 0, 7, 0, 0, 0, 199, 103, 71, 249,..."
2,CL01,CL0104,CL,Gaillon-sur-Montcient,Absence,Piste bidirect,1.0,1.0,121.30000,LT,...,None,None,0.0,0.0,121.3,0.0,53,None,121.338843,"[1, 2, 0, 0, 0, 10, 0, 0, 0, 194, 178, 188, 57..."
69,CL01,CL0101,CL,Tessancourt,Absence,Voie verte,1.0,0.0,413.60000,LT,...,None,None,413.6,1.0,0.0,225.0,274,None,18.507138,"[1, 2, 0, 0, 0, 2, 0, 0, 0, 86, 103, 250, 88, ..."


In [74]:
ddb.sql("""
select * from 's3://dlb-hackathon/datasets-diffusion/donnees_territoire_GPSEO/stationnements_trottinettes/STATIONNEMENT_TROTTINETTE_CU.parquet'
""").to_df().sample(5)

,objectid,id,nom,nom_statio,commune,index,code_insee,categorie,sous_categ,equipement,...,longueur_m,largeur_m,superficie,date_mise_,nom_create,date_creat,nom_editeu,date_maj,commentair,geom
2,8,CSP17,Maillage,CSP17 Route d'Andrésy,Carrières-sous-Poissy,Ca-31,78123,Maillage,Maillage,Maillage,...,5.0,2.0,10.0,02/05/2023,None,None,None,None,None,"[1, 1, 0, 0, 0, 129, 147, 64, 90, 128, 86, 0, ..."
317,320,AND05,AND05 Leclerc,AND05 Leclerc,Andrésy,None,78015,None,None,None,...,5.0,2.0,10.0,26/01/2024,None,None,None,None,None,"[1, 1, 0, 0, 0, 209, 79, 132, 223, 174, 93, 0,..."
164,169,LIM14,LIM14 Briand,LIM14 Briand,Limay,None,78335,None,None,None,...,3.5,2.0,7.0,26/01/2024,None,None,None,None,None,"[1, 1, 0, 0, 0, 50, 39, 134, 0, 65, 207, 251, ..."
299,302,POI21,Centre de PMI,POI21 Frémont,Poissy,Po-26,78498,Services d'accompagnement social,Services d'assistance sociale et familiale,Centre de PMI,...,2.0,2.0,4.0,02/05/2023,None,None,None,None,None,"[1, 1, 0, 0, 0, 76, 214, 179, 55, 21, 73, 0, 6..."
167,172,LIM24,LIM24 Vinaver,LIM24 Vinaver 1,Limay,None,78335,None,None,None,...,2.5,2.0,5.0,26/01/2024,None,None,None,None,None,"[1, 1, 0, 0, 0, 207, 129, 43, 159, 188, 153, 2..."


In [75]:
ddb.sql("""
select * from 's3://dlb-hackathon/datasets-diffusion/donnees_territoire_GPSEO/stationnements_velos/MOBILITE_VELO_STATIONNEMENT_CU.parquet'
""").to_df().sample(5)

,objectid,osm_id,source,geom
335,350,None,GPSEO,"[1, 1, 0, 0, 0, 154, 117, 230, 79, 107, 154, 2..."
77,72,n3241154578,None,"[1, 1, 0, 0, 0, 195, 8, 235, 164, 189, 65, 0, ..."
505,543,None,GPSEO,"[1, 1, 0, 0, 0, 14, 167, 225, 81, 218, 27, 255..."
346,365,None,GPSEO,"[1, 1, 0, 0, 0, 243, 132, 7, 178, 157, 239, 25..."
4,197,None,None,"[1, 1, 0, 0, 0, 159, 98, 255, 161, 69, 54, 0, ..."


# Historique état des ascenseurs

# Historique de fonctionnement des ascenseurs
## Description générale
Ce fichier correspond à l'agrégation des relevés de fonctionnement d'ascenseurs localisés dans les gares et stations gérées par la RATP et la SNCF en Ile de France.
Ces données comportent un historique allant de début 2012 à fin octobre 2024

Une version en ligne de ces données est disponible dans une forme non historisée (actualisation 3 fois par jour) :
- https://data.iledefrance-mobilites.fr/explore/dataset/etat-des-ascenseurs/information/
- https://eu.ftp.opendatasoft.com/stif/Ascenseurs/2022-IDFM-etat-de-ascenseurs.pdf

## Couverture temporelle 
|operateur|min(date_releve)|max(date_releve)|nb_appareils|nb_releves|nb_fichiers|
|---------|----------------|----------------|------------|----------|-----------|
|RATP|2021-01-01 08:07:57.000|2024-10-28 19:08:38.000|388|1004236|3332|
|SNCF|2022-01-08 14:34:28.000|2024-10-28 19:03:22.000|568|496914|1019|


In [88]:
ddb.sql(""" 
select * from 's3://dlb-hackathon/datasets-diffusion/ascenseurs_historique_etat/RELEVES_ETATS_ASCENSEURS_SNCF_RATP_2021-2024.parquet'
 """).to_df()

,operateur,date_releve,code_appareil,code_fonctionnel,etat,date_etat_debut,date_etat_fin,emplacement_gare,emplacement_situation,emplacement_direction
0,RATP,2021-01-01 08:07:57,1004,ASC01-EQ11-A005-000000269,2,01/01/2021,None,SAINT OUEN,SA200,None
1,RATP,2021-01-01 08:07:57,1005,ASC02-EQ11-A005-000000270,2,01/01/2021,02/01/2021,SAINT OUEN,SA200,None
2,RATP,2021-01-01 08:07:57,1006,ASC03-EQ11-A005-000000271,2,01/01/2021,None,SAINT OUEN,SA200,None
3,RATP,2021-01-01 08:07:57,1007,ASC04-EQ11-A005-000000272,2,01/01/2021,None,SAINT OUEN,SA200,None
4,RATP,2021-01-01 08:07:57,1008,ASC05-EQ11-A005-000000273,2,01/01/2021,02/01/2021,SAINT OUEN,SA200,None
...,...,...,...,...,...,...,...,...,...,...
1501145,SNCF,2024-10-28 19:03:22,983,2065,1,28/10/2024,None,EPINAY VILLETAN,Quai A<>Hall BV,Paris
1501146,SNCF,2024-10-28 19:03:22,984,2066,1,28/10/2024,None,EPINAY VILLETAN,Quai B,Paris/Province
1501147,SNCF,2024-10-28 19:03:22,985,2067,1,28/10/2024,None,EPINAY VILLETAN,Quai C,Province
1501148,SNCF,2024-10-28 19:03:22,986,2068,1,28/10/2024,None,EPINAY VILLETAN,Quai Arcade,Province


# Historique des données de validation sur réseau ferré 2015 - 2024 (S1)

# Historique des données de validation 2015 - 2024 (S1)

## Description générale

Ce dataset comprend deux fichiers historisant les données de validation du **réseau ferré** telles que celles publiées par IDFM.
Voir la publication et documentation associée ici : https://prim.iledefrance-mobilites.fr/fr/jeux-de-donnees/validations-reseau-surface-nombre-validations-par-jour-1er-trimestre

- Données du 01/01/2021 au 30/06/2024 

## Distinction des formats 
- S1_2015_S1_2024_PROFIL_FER.parquet : nombre de validation par arrêt, type de jour et tranche horaire
- S1_2015_S1_2024_NB_FER.parquet : nombre de validations par jour

Les données au format csv sont aussi disponibles dans les sous-répertoires.

In [89]:
ddb.sql(""" 
select * from 's3://dlb-hackathon/datasets-diffusion/validations_historisees/S1_2015_S1_2024_PROFIL_FER.parquet'
 """).to_df()

,semestre,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_REFA_LDA,CAT_JOUR,TRNC_HORR_60,pourc_validations
0,2015_S1,100,110,1,PORTE MAILLOT,71379,DIJFP,14H-15H,6.54
1,2015_S1,100,110,1,PORTE MAILLOT,71379,DIJFP,17H-18H,8.37
2,2015_S1,100,110,1,PORTE MAILLOT,71379,JOHV,0H-1H,0.65
3,2015_S1,100,110,1,PORTE MAILLOT,71379,JOHV,13H-14H,3.73
4,2015_S1,100,110,1,PORTE MAILLOT,71379,JOHV,16H-17H,5.48
...,...,...,...,...,...,...,...,...,...
1484638,2024_S1,810,802,905,VILLEPINTE,None,JOHV,12H-13H,"3,87"
1484639,2024_S1,810,802,905,VILLEPINTE,None,JOHV,17H-18H,"5,36"
1484640,2024_S1,810,802,905,VILLEPINTE,None,JOVS,23H-0H,"0,06"
1484641,2024_S1,810,802,905,VILLEPINTE,None,JOVS,7H-8H,"15,58"


In [90]:
ddb.sql(""" 
select * from 's3://dlb-hackathon/datasets-diffusion/validations_historisees/S1_2015_S1_2024_NB_FER.parquet'
 """).to_df()

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_REFA_LDA,CATEGORIE_TITRE,NB_VALD
0,2017-01-01,100,110,100,BOULOGNE-PONT DE SAINT CLOUD,70721,IMAGINE R,225
1,2017-01-01,100,110,1006,OLYMPIADES,71557,AMETHYSTE,180
2,2017-01-01,100,110,1007,LES AGNETTES-ASNIERES-GENNEVILLIERS,72240,FGT,113
3,2017-01-01,100,110,104,BOURSE,73635,AMETHYSTE,40
4,2017-01-01,100,110,104,BOURSE,73635,TST,15
...,...,...,...,...,...,...,...,...
17442756,2024-06-30,810,802,795,SEVRAN BEAUDOTT,None,Imagine R,772
17442757,2024-06-30,810,802,827,SAINT-MICHEL,None,Amethyste,106
17442758,2024-06-30,810,802,889,VERT GALANT,None,Forfait Navigo,1842
17442759,2024-06-30,810,802,889,VERT GALANT,None,Imagine R,644


In [97]:
# Exemple d'analyse thématique
ddb.sql(""" 
select substring(jour::varchar, 0,5) annee , sum(case when lower(nb_vald) = upper(nb_vald) then nb_vald else null end::int) nb_validations from 's3://dlb-hackathon/datasets-diffusion/validations_historisees/S1_2015_S1_2024_NB_FER.parquet' group by all
order by 1 """)

┌─────────┬────────────────┐
│  annee  │ nb_validations │
│ varchar │     int128     │
├─────────┼────────────────┤
│ 2015    │     1493460776 │
│ 2016    │     1589563064 │
│ 2017    │     1671982464 │
│ 2018    │     1675803004 │
│ 2019    │     1695721547 │
│ 2020    │      926003724 │
│ 2021    │     1251011175 │
│ 2022    │     1628679804 │
│ 2023    │     1726555186 │
│ 2024    │      948293948 │
├─────────┴────────────────┤
│ 10 rows        2 columns │
└──────────────────────────┘